In [287]:
from google.colab import files

# Upload training and testing data
print("Upload training")
uploaded_train = files.upload()
print("Upload testing")
uploaded_test = files.upload()

Upload training


Saving Data_Training_JST.csv to Data_Training_JST (13).csv
Upload testing


Saving Data_Testing_JST.csv to Data_Testing_JST (13).csv


In [288]:
# Fungsi untuk membuat tabel saat membaca data

#Source: https://www.geeksforgeeks.org/how-to-render-pandas-dataframe-as-html-table/
from IPython.display import display, HTML

def tabel(df):
    display(HTML(df.to_html(index=False)))

In [289]:
# Membaca dan Menampilkan data training dan data testing
import pandas as pd

#Data Training
# Membaca
train_data = pd.read_csv(next(iter(uploaded_train.keys())))
# Menampilkan
print("Data Training:")
# Function Tabel untuk menampilkan dalam bentuk tabel
tabel(train_data)

# Data Testing
# Membaca
test_data = pd.read_csv(next(iter(uploaded_test.keys())),on_bad_lines='skip') # saat tidak memakai variabel "on_bad_lines" akan terjadi error, source penyelesaian: https://stackoverflow.com/questions/18039057/pandas-parser-cparsererror-error-tokenizing-data
# Menampilkan
print("Data Testing:")
# Function Tabel untuk menampilkan dalam bentuk tabel
tabel(test_data)

Data Training:


Lebar Mulut/Jarak Pupil Mata,Lebar Dagu/Jarak Pupil Mata,Lebar Pipi/Jarak Pupil Mata,Lebar Dahi/Jarak Pupil Mata,Lebar Hidung/Jarak Pupil Mata,Lebar Mata/Jarak Pupil Mata,Tinggi Hidung/Jarak Pupil Mata,Label
1.30,1.20,2.10,2.00,0.85,1.65,1.00,B.J.Habibie
0.78,0.93,1.56,1.33,0.58,1.22,0.78,B.J.Habibie
0.76,0.80,1.67,1.48,0.57,1.28,0.83,B.J.Habibie
0.83,1.00,1.64,1.55,0.60,1.36,0.71,B.J.Habibie
0.75,1.07,1.43,1.52,0.55,1.25,0.80,B.J.Habibie
0.83,1.00,1.63,1.25,0.63,1.31,0.83,B.J.Habibie
0.67,1.00,1.44,1.56,0.51,1.33,0.78,B.J.Habibie
0.77,0.86,1.77,1.57,0.57,1.29,0.86,B.J.Habibie
0.80,0.89,1.60,1.56,0.64,1.33,0.78,B.J.Habibie
0.83,0.83,1.83,2.00,0.67,1.23,1.00,B.J.Habibie


Data Testing:


Lebar Mulut/Jarak Pupil Mata,Lebar Dagu/Jarak Pupil Mata,Lebar Pipi/Jarak Pupil Mata,Lebar Dahi/Jarak Pupil Mata,Lebar Hidung/Jarak Pupil Mata,Lebar Mata/Jarak Pupil Mata,Tinggi Hidung/Jarak Pupil Mata,Label
0.86,0.97,1.71,1.43,0.71,1.34,0.97,B.J.Habibie
0.73,0.90,1.68,1.46,0.61,1.22,0.78,B.J.Habibie
0.74,0.64,1.70,1.38,0.64,1.28,0.74,B.J.Habibie
0.76,0.80,1.52,1.40,0.58,1.24,0.80,B.J.Habibie
0.79,0.83,1.50,1.43,0.60,1.21,0.81,B.J.Habibie
0.86,0.95,1.79,1.43,0.67,1.40,0.95,Soeharto
0.80,0.95,1.67,1.43,0.63,1.35,0.87,Soeharto
0.76,0.86,1.57,1.30,0.67,1.40,0.86,Soeharto
0.87,1.09,1.74,1.74,0.65,1.43,1.00,Soeharto
0.91,0.91,1.59,1.59,1.09,1.43,0.98,Soeharto


Preprocessing Data

In [290]:
# Pisahkan ciri dan label training
# Referensi ".iloc": https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html
X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].astype(str).values

In [291]:
# Pisahkan ciri dan label testing
# Referensi ".iloc": https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html
X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].astype(str).values

In [292]:
# Function encode label
import numpy as np
def encode_labels(labels):
    encoded_labels = []
    for label in labels:
        if label == 'B.J.Habibie': #y_input
            encoded_labels.append(0.86770)
        else:
            encoded_labels.append(0)
    return np.array(encoded_labels)

In [293]:
# encode label tiap jenis data
y_train = encode_labels(y_train)
y_test = encode_labels(y_test)

In [294]:
y_train

array([0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.8677,
       0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.8677,
       0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.8677,
       0.8677])

In [295]:
y_test

array([0.8677, 0.8677, 0.8677, 0.8677, 0.8677, 0.    , 0.    , 0.    ,
       0.    , 0.    ])

JST

In [296]:
# Inisialisasi parameter JST
input_neurons = X_train.shape[1]
hidden_neurons = int(2/3 * input_neurons)
output_neurons = 1

In [297]:
# Inisialisasi bobot

# Bobot dari input ke hidden
weights_input = np.array([[0.2, 0.3, 0.4, 0.5],
                          [0.3, 0.4, 0.5, 0.6],
                          [0.4, 0.5, 0.6, 0.7],
                          [0.5, 0.6, 0.7, 0.8],
                          [0.6, 0.7, 0.8, 0.9],
                          [0.7, 0.8, 0.9, 0.3],
                          [0.8, 0.9, 0.3, 0.5]])

# Bobot dari hidden ke output
weights_output = np.array([[0.3], [0.4], [0.5], [0.6]])

In [298]:
# Inisialisasi bias
b1 = 0.05
b2 = 0.1

In [299]:
# Fungsi aktivasi h_output
def h_output(x):
    return 1 / (1 + np.exp(-x))

In [300]:
# Fungsi untuk forward pass
def forward_pass(X):
    net_h = np.dot(X, weights_input) + b1 * 1
    out_h = h_output(net_h)
    net_o = np.dot(out_h, weights_output) + b2 * 1
    out_o = h_output(net_o)
    return net_h, out_h, net_o, out_o

In [301]:
# Hiperparameter
eta = 0.5
max_perulangan = 3

In [302]:
# Turunan H_output terhadap Net_output
def TurunanH_OutputTerhadapNetO(x):
    return x * (1 - x)

In [303]:
# Referensi: https://www.geeksforgeeks.org/backpropagation-in-neural-network/
# Fungsi untuk backward pass
def backward_pass(X, y, net_h, out_h, net_o, out_o):
    global weights_output, weights_input  # Menggunakan variabel global untuk bobot output dan input

    # Menghitung total error: setengah dari kuadrat selisih antara nilai target (y) dan output prediksi (out_o)
    total_error = 0.5 * (y - out_o) ** 2

    # Turunan total error terhadap output: negative gradient dari fungsi error
    a_total_error_d_out_o = -(y - out_o)

    # Turunan output terhadap net output: turunan dari fungsi aktivasi output
    a_out_o_d_net_o = TurunanH_OutputTerhadapNetO(out_o)

    # Turunan net output terhadap bobot output: hidden layer output transposed (d_net_o/d_w)
    a_net_o_d_w_output = out_h.T

    # Turunan total error terhadap bobot output: gradien total error terhadap bobot output
    a_total_error_d_w_output = a_total_error_d_out_o * a_out_o_d_net_o * a_net_o_d_w_output

    # Turunan net output terhadap hidden output: bobot output transposed
    a_net_o_d_out_h = weights_output.T

    # Turunan total error terhadap net hidden: gradien total error terhadap net hidden
    a_total_error_d_net_h = (a_total_error_d_out_o * a_out_o_d_net_o).dot(a_net_o_d_out_h)

    # Turunan hidden output terhadap net hidden: turunan dari fungsi aktivasi hidden
    a_out_h_d_net_h = out_h * (1 - out_h)

    # Turunan total error terhadap bobot input: gradien total error terhadap bobot input
    a_total_error_d_w_input = X.T.dot(a_total_error_d_net_h * a_out_h_d_net_h)

    # Memperbarui bobot output: mengurangi bobot output dengan gradien yang dihitung, diskalakan oleh learning rate (eta)
    weights_output -= eta * a_total_error_d_w_output

    # Memperbarui bobot input: mengurangi bobot input dengan gradien yang dihitung, diskalakan oleh learning rate (eta)
    weights_input -= eta * a_total_error_d_w_input

    # Mengembalikan total error sebagai hasil dari fungsi backward pass
    return total_error

In [304]:
# Training JST
for i in range(len(X_train)):
    print("==========================================================================================================")
    print(f"Data Latih ke-{i + 1}")
    forward_backward_repeats = 0
    while forward_backward_repeats < max_perulangan:
        # Cari nilai net_h, out_h, net_o, out_o
        net_h, out_h, net_o, out_o = forward_pass(X_train[i].reshape(1, -1))

        # Ambang Batas
        predicted_label = out_o

        # Menampilkan hasil forward pass
        print(f"Forward-backward pass {forward_backward_repeats + 1}")
        print(f"net_h: {net_h}")
        print(f"out_h: {out_h}")
        print(f"net_o: {net_o}")
        print(f"out_o: {out_o}")
        print(f"y_input: 0.8677")
        print(f"f(x): {out_o}")
        print(f"bobot input: {weights_input}")
        print(f"bobot output: {weights_output}")

        # Ambang Batas
        if predicted_label >= y_train[i]:
            print("Forward Pass ke foto selanjutnya.")
            break
        else:
            print(f"Backward Pass karena y_input = {out_o} < {y_train[i]}.")
            total_error = backward_pass(X_train[i].reshape(1, -1), y_train[i].reshape(1, -1), net_h, out_h, net_o, out_o)

            # Menampilkan hasil backward pass
            print(f"Total Error: {total_error}")
            print(f"Update Bobot input: {weights_input}")
            print(f"Updated Bobot output: {weights_output}")
            forward_backward_repeats += 1
            if forward_backward_repeats == max_perulangan:
                print("Skip, Foto Selanjutnya")
    print()

Data Latih ke-1
Forward-backward pass 1
net_h: [[4.975 5.985 6.295 6.25 ]]
out_h: [[0.99313888 0.9974901  0.99815789 0.99807327]]
net_o: [[1.89486061]]
out_o: [[0.86930874]]
y_input: 0.8677
f(x): [[0.86930874]]
bobot input: [[0.2 0.3 0.4 0.5]
 [0.3 0.4 0.5 0.6]
 [0.4 0.5 0.6 0.7]
 [0.5 0.6 0.7 0.8]
 [0.6 0.7 0.8 0.9]
 [0.7 0.8 0.9 0.3]
 [0.8 0.9 0.3 0.5]]
bobot output: [[0.3]
 [0.4]
 [0.5]
 [0.6]]
Forward Pass ke foto selanjutnya.

Data Latih ke-2
Forward-backward pass 1
net_h: [[3.6   4.318 4.49  4.432]]
out_h: [[0.97340301 0.98684875 0.98890386 0.98824904]]
net_o: [[1.87416176]]
out_o: [[0.86693909]]
y_input: 0.8677
f(x): [[0.86693909]]
bobot input: [[0.2 0.3 0.4 0.5]
 [0.3 0.4 0.5 0.6]
 [0.4 0.5 0.6 0.7]
 [0.5 0.6 0.7 0.8]
 [0.6 0.7 0.8 0.9]
 [0.7 0.8 0.9 0.3]
 [0.8 0.9 0.3 0.5]]
bobot output: [[0.3]
 [0.4]
 [0.5]
 [0.6]]
Backward Pass karena y_input = [[0.86693909]] < 0.8677.
Total Error: [[2.8948927e-07]]
Update Bobot input: [[0.20000027 0.30000018 0.40000019 0.50000024]
 [0.30000

In [305]:
# Training JST
with open('training.txt', 'w') as train_file:
    for i in range(len(X_train)):
        train_file.write("==========================================================================================================\n")
        train_file.write(f"Data Latih ke-{i + 1}\n")
        forward_backward_repeats = 0
        while forward_backward_repeats < max_perulangan:
            # Cari nilai net_h, out_h, net_o, out_o
            net_h, out_h, net_o, out_o = forward_pass(X_train[i].reshape(1, -1))

            # Ambang Batas
            predicted_label = out_o

            # Menampilkan hasil forward pass
            train_file.write(f"Forward-backward pass {forward_backward_repeats + 1}\n")
            train_file.write(f"net_h: {net_h}\n")
            train_file.write(f"out_h: {out_h}\n")
            train_file.write(f"net_o: {net_o}\n")
            train_file.write(f"out_o: {out_o}\n")
            train_file.write("y_input: 0.8677\n")
            train_file.write(f"f(x): {out_o}\n")
            train_file.write(f"bobot input: {weights_input}\n")
            train_file.write(f"bobot output: {weights_output}\n")

            # Ambang Batas
            if predicted_label >= y_train[i]:
                train_file.write("Forward Pass ke foto selanjutnya.\n")
                break
            else:
                train_file.write(f"Backward Pass karena y_input = {out_o} < {y_train[i]}.\n")
                total_error = backward_pass(X_train[i].reshape(1, -1), y_train[i].reshape(1, -1), net_h, out_h, net_o, out_o)

                # Menampilkan hasil backward pass
                train_file.write(f"Total Error: {total_error}\n")
                train_file.write(f"Update Bobot input: {weights_input}\n")
                train_file.write(f"Updated Bobot output: {weights_output}\n")
                forward_backward_repeats += 1
        train_file.write("\n")

In [306]:
# Pengujian Model
correct_predictions = 0
print("Testing Data Results:")
for i in range(len(X_test)):
    net_h, out_h, net_o, out_o = forward_pass(X_test[i].reshape(1, -1))
    predicted_label = out_o

    # Menampilkan hasil perhitungan
    print(f"Data Testing Ke-{i + 1}")
    print(f"net_h: {net_h}")
    print(f"out_h: {out_h}")
    print(f"net_o: {net_o}")
    print(f"out_o: {out_o}")
    print(f"y_input: 0.8677")
    print(f"f(x): {out_o}")
    print(f"bobot input: {weights_input}")
    print(f"bobot output: {weights_output}")


    if predicted_label >= y_test[i]:
        correct_predictions += 1
        print(f"Hasil pengenalan foto ke-{i+1}: Benar")
    else:
        print(f"Hasil pengenalan foto ke-{i+1}: Salah")
    print()

Testing Data Results:
Data Testing Ke-1
net_h: [[4.0520851  4.85105701 4.97106042 4.92907723]]
out_h: [[0.98291103 0.99224057 0.99311198 0.99281877]]
net_o: [[1.88879278]]
out_o: [[0.86861782]]
y_input: 0.8677
f(x): [[0.86861782]]
bobot input: [[0.20000662 0.30000443 0.4000047  0.500006  ]
 [0.30000863 0.40000578 0.50000613 0.60000784]
 [0.40001505 0.50001008 0.60001069 0.70001366]
 [0.50001331 0.60000892 0.70000944 0.80001207]
 [0.60000505 0.70000338 0.80000359 0.90000458]
 [0.70001159 0.80000776 0.90000823 0.30001052]
 [0.80000737 0.90000494 0.30000524 0.50000669]]
bobot output: [[0.30119247]
 [0.40120876]
 [0.50121122]
 [0.60121033]]
Hasil pengenalan foto ke-1: Benar

Data Testing Ke-2
net_h: [[3.71208028 4.45005379 4.642057   4.60407286]]
out_h: [[0.97615578 0.98845686 0.99045415 0.99008825]]
net_o: [[1.88226633]]
out_o: [[0.86787123]]
y_input: 0.8677
f(x): [[0.86787123]]
bobot input: [[0.20000662 0.30000443 0.4000047  0.500006  ]
 [0.30000863 0.40000578 0.50000613 0.60000784]
 [0.

In [307]:
# Pengujian Model
correct_predictions = 0
with open('testing.txt', 'w') as test_file:
    test_file.write("Testing Data Results:\n")
    for i in range(len(X_test)):
        net_h, out_h, net_o, out_o = forward_pass(X_test[i].reshape(1, -1))
        predicted_label = out_o

        # Menampilkan hasil perhitungan
        test_file.write(f"Data Testing Ke-{i + 1}\n")
        test_file.write(f"net_h: {net_h}\n")
        test_file.write(f"out_h: {out_h}\n")
        test_file.write(f"net_o: {net_o}\n")
        test_file.write(f"out_o: {out_o}\n")
        test_file.write("y_input: 0.8677\n")
        test_file.write(f"f(x): {out_o}\n")

        if predicted_label >= y_test[i]:
            correct_predictions += 1
            test_file.write("Hasil pengenalan foto: Benar\n")
        else:
            test_file.write("Hasil pengenalan foto: Salah\n")
        test_file.write("\n")


In [308]:
# Hitung akurasi
accuracy = correct_predictions / len(y_test)
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.7000
